<a href="https://colab.research.google.com/github/shivakumarsanugula/Kaggle-NLP/blob/main/Fake_News_Classifier_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd

In [10]:
df = pd.read_csv('train.csv',error_bad_lines=False,engine='python')

<ipython-input-10-235c2076de0a>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('train.csv',error_bad_lines=False,engine='python')
Skipping line 6436: unexpected end of data


In [11]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [12]:
## check null
df.isnull().sum()

id          0
title     178
author    619
text       13
label       0
dtype: int64

In [14]:
## Drop Nan values
df = df.dropna()

In [15]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [16]:
# Get the independent Features

X = df.drop('label',axis=1)

In [17]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [18]:
## Get the Dependent Feature
y = df['label']

In [19]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [20]:
X.shape

(5637, 4)

In [21]:
y.shape

(5637,)

In [22]:
import tensorflow as tf

In [23]:
tf.__version__

'2.14.0'

In [24]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [25]:
# Vocabulary size
voc_size = 5000

Onehot Representation

In [26]:
messages = X.copy()

In [27]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [28]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
6429,6429,Moveable Feast Cafe 2016/10/28 … Open Thread |...,Herb Swanson,Leave a Reply Click here to get more info on f...
6430,6430,"Hajj Prep: Search Soul, Buy Sturdy Shoes, Pay ...",Diaa Hadid,"JIDDA, Saudi Arabia — My father was on the ..."
6431,6431,"10 radikale Vorhaben, die Donald Trump sofort ...",noreply@blogger.com (Der Postillon),"Donnerstag, 10. November 2016 10 radikale Vorh..."
6432,6432,Little Blue Balls...,recrisp,link Years ago I came here for the UFOs and af...


In [30]:
messages.reset_index(inplace=True)

In [31]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
5632,6429,6429,Moveable Feast Cafe 2016/10/28 … Open Thread |...,Herb Swanson,Leave a Reply Click here to get more info on f...
5633,6430,6430,"Hajj Prep: Search Soul, Buy Sturdy Shoes, Pay ...",Diaa Hadid,"JIDDA, Saudi Arabia — My father was on the ..."
5634,6431,6431,"10 radikale Vorhaben, die Donald Trump sofort ...",noreply@blogger.com (Der Postillon),"Donnerstag, 10. November 2016 10 radikale Vorh..."
5635,6432,6432,Little Blue Balls...,recrisp,link Years ago I came here for the UFOs and af...


In [32]:
import nltk
import re
from nltk.corpus import stopwords

In [33]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [34]:
## Dataset Preprocessing

from nltk.stem.porter import PorterStemmer ## stemming purpose
ps = PorterStemmer()
corpus = []

for i in range(0, len(messages)):
  #print(i)
  review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [35]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [36]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [37]:
onehot_rep = [one_hot(words,voc_size) for words in corpus]
onehot_rep

[[2360, 649, 1581, 3652, 2503, 4433, 1218, 2121, 2237, 4992],
 [2936, 3912, 2978, 4588, 4623, 1633, 1092],
 [1780, 3706, 4473, 3446],
 [1886, 4869, 2143, 2847, 3864, 1203],
 [400, 4623, 2690, 640, 553, 2866, 4623, 1682, 981, 230],
 [2854,
  4414,
  2039,
  361,
  4765,
  575,
  2276,
  829,
  1154,
  4809,
  3230,
  3937,
  3870,
  757,
  1092],
 [3933, 4436, 1752, 2547, 3592, 3939, 3278, 3014, 2043, 2329, 475],
 [507, 2638, 3839, 4458, 324, 1737, 575, 1433, 2043, 2329, 475],
 [4596, 3768, 3720, 2449, 2319, 1915, 4462, 2364, 575, 1896],
 [4118, 3598, 985, 3014, 4778, 316, 2857, 863],
 [3404, 2246, 1224, 1487, 802, 3926, 2990, 16, 3972, 3624, 530],
 [2847, 3876, 2503, 1915, 575, 324],
 [4262, 41, 3973, 3895, 2026, 4658, 2164, 2198, 310],
 [980, 3855, 1104, 2866, 4976, 919, 3892, 2043, 2329, 475],
 [10, 164, 3369, 1834, 2788, 2043, 2329, 475],
 [1404, 128, 2219, 4860, 2130, 1609, 3698, 4695, 2118, 1106],
 [3053, 2809, 3912],
 [2625, 2192, 4921, 3028, 575, 2701, 736, 1092],
 [3240, 4144, 

In [39]:
print(corpus[1])
onehot_rep[1]

flynn hillari clinton big woman campu breitbart


[2936, 3912, 2978, 4588, 4623, 1633, 1092]

## Embedding Representation

In [45]:
# the length of all the sentences should be same,in order to make same length we will do Padding
sent_length = 20
embedded_docs = pad_sequences(onehot_rep,padding='post',maxlen=sent_length)
print(embedded_docs)

[[2360  649 1581 ...    0    0    0]
 [2936 3912 2978 ...    0    0    0]
 [1780 3706 4473 ...    0    0    0]
 ...
 [3616 1406  740 ...    0    0    0]
 [ 933 4387  455 ...    0    0    0]
 [4554 3509 3693 ...    0    0    0]]


In [46]:
print(corpus[1])
print(onehot_rep[1])
embedded_docs[1]

flynn hillari clinton big woman campu breitbart
[2936, 3912, 2978, 4588, 4623, 1633, 1092]


array([2936, 3912, 2978, 4588, 4623, 1633, 1092,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [49]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [50]:
len(embedded_docs),y.shape

(5637, (5637,))

In [51]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [52]:
X_final.shape,y_final.shape

((5637, 20), (5637,))

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=42)

# Model Training

In [54]:
## Training the model
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
59/59 [==============================] - 6s 60ms/step - loss: 0.5303 - accuracy: 0.6976 - val_loss: 0.3017 - val_accuracy: 0.8829
Epoch 2/10
59/59 [==============================] - 4s 66ms/step - loss: 0.1675 - accuracy: 0.9351 - val_loss: 0.2364 - val_accuracy: 0.9044
Epoch 3/10
59/59 [==============================] - 2s 42ms/step - loss: 0.0648 - accuracy: 0.9788 - val_loss: 0.3773 - val_accuracy: 0.8812
Epoch 4/10
59/59 [==============================] - 2s 42ms/step - loss: 0.0219 - accuracy: 0.9944 - val_loss: 0.3867 - val_accuracy: 0.8915
Epoch 5/10
59/59 [==============================] - 2s 42ms/step - loss: 0.0091 - accuracy: 0.9981 - val_loss: 0.5082 - val_accuracy: 0.8850
Epoch 6/10
59/59 [==============================] - 2s 42ms/step - loss: 0.0058 - accuracy: 0.9989 - val_loss: 0.5821 - val_accuracy: 0.8812
Epoch 7/10
59/59 [==============================] - 4s 75ms/step - loss: 0.0041 - accuracy: 0.9995 - val_loss: 0.5832 - val_accuracy: 0.8845
Epoch 8/10
59

In [64]:
## Adding Dropout ---> basically it is improvement of the model
from tensorflow.keras.layers import Dropout
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout_1 (Dropout)         (None, 20, 40)            0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [65]:
## Training the model
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
59/59 [==============================] - 5s 47ms/step - loss: 0.5364 - accuracy: 0.7111 - val_loss: 0.3152 - val_accuracy: 0.8619
Epoch 2/10
59/59 [==============================] - 3s 43ms/step - loss: 0.1932 - accuracy: 0.9227 - val_loss: 0.2386 - val_accuracy: 0.9001
Epoch 3/10
59/59 [==============================] - 3s 60ms/step - loss: 0.1048 - accuracy: 0.9653 - val_loss: 0.2560 - val_accuracy: 0.9006
Epoch 4/10
59/59 [==============================] - 4s 60ms/step - loss: 0.0567 - accuracy: 0.9804 - val_loss: 0.3788 - val_accuracy: 0.8963
Epoch 5/10
59/59 [==============================] - 3s 44ms/step - loss: 0.0306 - accuracy: 0.9913 - val_loss: 0.3658 - val_accuracy: 0.8979
Epoch 6/10
59/59 [==============================] - 3s 44ms/step - loss: 0.0282 - accuracy: 0.9923 - val_loss: 0.3867 - val_accuracy: 0.8823
Epoch 7/10
59/59 [==============================] - 3s 44ms/step - loss: 0.0194 - accuracy: 0.9952 - val_loss: 0.4960 - val_accuracy: 0.8877
Epoch 8/10
59

# Performance Metrices And Accuracy

In [66]:
y_pred=model.predict(X_test)

59/59 [==============================] - 1s 10ms/step


In [67]:
y_pred=np.where(y_pred > 0.5,1,0)

In [68]:
from sklearn.metrics import confusion_matrix

In [69]:
confusion_matrix(y_test,y_pred)

array([[968,  89],
       [114, 690]])

In [70]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.890918860827512

In [71]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.89      0.92      0.91      1057
           1       0.89      0.86      0.87       804

    accuracy                           0.89      1861
   macro avg       0.89      0.89      0.89      1861
weighted avg       0.89      0.89      0.89      1861

